understat - 2015-2024 Matches + Season team stats

In [ ]:
import pandas as pd
import ast
from understatapi import UnderstatClient

In [ ]:
# Configuration parameters
START_YEAR = 2015
END_YEAR = 2024

In [ ]:
# Initialize Understat client
client = UnderstatClient()

In [ ]:
# Initialize lists to store data
match_data_all = []
team_data_all = []

In [ ]:
# Fetch match data for each season
for year in range(START_YEAR, END_YEAR + 1):
    try:
        # Fetch match data for the season
        match_data = client.league('EPL').get_match_data(season=str(year))
        
        # Add season info to each match
        for match in match_data:
            match['season'] = year
        
        # Add to overall collection
        match_data_all.extend(match_data)
        
    except Exception as e:
        print(f"   Error fetching match data for {year}: {e}")
        continue

df_matches = pd.json_normalize(match_data_all, sep='_')
# print(df_matches.head())

In [ ]:
# Fetch team data for each season
for year in range(START_YEAR, END_YEAR + 1):
    
    try:
        # Fetch team data for the season
        team_data = client.league('EPL').get_team_data(season=str(year))
        
        # Process team data
        for team_name, stats in team_data.items():
            stats['season'] = year
            stats['team_name'] = team_name
            team_data_all.append(stats)
        
    except Exception as e:
        print(f"   Error fetching team data for {year}: {e}")
        continue

df_teams = pd.DataFrame(team_data_all)
# print(df_teams.head())

In [ ]:
# Match team IDs to home match titles to ensure correct team names.
id_title_map = (
    df_matches[['h_id', 'h_title']]
    .dropna()
    .drop_duplicates(subset='h_id')
    .set_index('h_id')['h_title']
    .to_dict()
)

df_teams['title'] = df_teams['id'].map(id_title_map)

In [ ]:
summary_rows = []

for _, row in df_teams.iterrows():
    team_id = row.get('id')
    title = row.get('title')
    season = row.get('season')
    history = row.get('history')

    if isinstance(history, str):
        history = ast.literal_eval(history)

    total_goals = sum(match.get('scored', 0) for match in history)
    total_conceded = sum(match.get('missed', 0) for match in history)
    total_xG = sum(match.get('xG', 0) for match in history)
    total_xGA = sum(match.get('xGA', 0) for match in history)
    total_pts = sum(match.get('pts', 0) for match in history)
    total_xpts = sum(match.get('xpts', 0) for match in history)

    summary_rows.append({
        'id': team_id,
        'title': title,
        'season': season,
        'goals': total_goals,
        'conceded': total_conceded,
        'xG': round(total_xG, 2),
        'xGA': round(total_xGA, 2),
        'pts': total_pts,
        'xpts': round(total_xpts, 2)
    })

df_summary = pd.DataFrame(summary_rows)

# Drop duplicates based on 'id', 'xG', and 'xGA'
df_summary = df_summary.drop_duplicates(subset=['id', 'xG', 'xGA'])
print(df_summary)

In [ ]:
print(df_matches.columns, df_summary.columns)

clubelo - Elo Rankings by team in matches df

In [ ]:
# Create a unique list of home team titles in df_matches
unique_h_titles = df_matches['h_title'].dropna().unique().tolist()
print('Unique home team titles:', unique_h_titles)



In [ ]:
import requests
from io import StringIO

In [ ]:
# Map of Understat team titles to ClubElo API names
final_team_name_map = {
    'Manchester City': 'ManCity',
    'Manchester United': 'ManUnited',
    'Wolverhampton Wanderers': 'Wolves',
    'West Bromwich Albion': 'WestBrom',
    'Newcastle United': 'Newcastle',
    'Sheffield United': 'SheffieldUnited',
    'Nottingham Forest': 'Forest',
    'Burnley': 'Burnley',
    'Brighton & Hove Albion': 'Brighton',
}

clubelo_data = {}
error_log = []

# Function to fetch data with retries
def fetch_clubelo_data(api_title, max_retries=3, backoff_factor=1.0):
    for attempt in range(max_retries):
        try:
            url = f"http://api.clubelo.com/{api_title}"
            response = requests.get(url, timeout=10)
            response.raise_for_status()

            if not response.text.strip():
                raise ValueError("Empty response body")

            df = pd.read_csv(StringIO(response.text))
            if df.empty:
                raise ValueError("DataFrame is empty")

            return df

        except Exception as e:
            if attempt < max_retries - 1:
                wait = backoff_factor * (2 ** attempt)
                time.sleep(wait)
            else:
                raise e

# Main data retrieval loop
for team_name in unique_h_titles:
    api_name = final_team_name_map.get(team_name, team_name.replace(" ", ""))

    try:
        df = fetch_clubelo_data(api_name)
        clubelo_data[team_name] = df
        print(f"Successfully retrieved data for: {team_name} (API: {api_name})")
    except Exception as e:
        error_log.append((team_name, api_name, str(e)))
        print(f"Failed to retrieve data for: {team_name} (API: {api_name}) — {e}")

# Save successfully retrieved data
if clubelo_data:
    all_data_df = pd.concat(
        clubelo_data.values(),
        keys=clubelo_data.keys(),
        names=['team', 'row']
    ).reset_index(level=0)
    all_data_df.to_csv('clubelo_all_teams.csv', index=False)
    print("Saved all ClubElo data to clubelo_all_teams.csv")
else:
    print("No ClubElo data to save.")

# Save error log
if error_log:
    error_df = pd.DataFrame(error_log, columns=["Team", "API_Name", "Error"])
    error_df.to_csv("clubelo_errors.csv", index=False)
    print("Saved error log to clubelo_errors.csv")
else:
    print("No errors encountered.")

In [ ]:
# Print a unique list of team titles in the combined clubelo DataFrame
unique_elo_titles = all_clubelo_df['team'].unique().tolist()
print('Unique team titles in clubelo DataFrame:', unique_elo_titles)

# drop all rows wher all_clubelo_df'To' column is before 2015
all_clubelo_df = all_clubelo_df[all_clubelo_df['To'] >= '2015-01-01']

In [ ]:
# Save df_matches and all_clubelo_df to CSVs in the data/raw directory
df_matches.to_csv('/workspaces/ml-prem-cash-machine/data/raw/understat_matches.csv', index=False)
df_summary.to_csv('/workspaces/ml-prem-cash-machine/data/raw/understat_teamseasons.csv', index=False)
all_clubelo_df.to_csv('/workspaces/ml-prem-cash-machine/data/raw/clubelo_all_teams.csv', index=False)
